In [13]:
import pandas as pd
import numpy as np
import re
import math
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import (Embedding, Conv1D, GlobalMaxPooling1D, Dense, GRU, Flatten, 
                                     Dropout, GlobalAveragePooling2D, Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications import EfficientNetB1

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer, snowball
from nltk.corpus import stopwords

import pickle

In [9]:
##Model sur le google drive https://drive.google.com/file/d/1KEyC6HixfbFdhYYxsS4vyi1EJPo85lIH/view?usp=drive_link

model = load_model("./bimodal.h5")


In [14]:
##Les fichiers pickle sont sur Github, folder data

with open("label_encoder.pickle", "rb") as handle:
    le = pickle.load(handle)

In [18]:
with open("tokenizer.pickle", "rb") as handle:
    tokenizer = pickle.load(handle)


In [10]:
image_path="./images/images/crop_300_train/image_664491_product_184739.jpg"
text_input='Jeremy Mc Grath Vs Pastrana'

In [11]:
def preprocess_image(image_path, resize=(200, 200)):
    im = tf.io.read_file(image_path)
    im = tf.image.decode_png(im, channels=3)
    im = tf.image.resize(im, resize, method='nearest')
    return im

def preprocess_text(text, tokenizer, max_len=100):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_len)
    return padded_sequence[0]  # Retirer np.expand_dims dans la partie de prédiction

In [19]:
# Prétraitement de l'image et du texte
image = preprocess_image(image_path)
text = preprocess_text(text_input, tokenizer)

In [21]:
# Ajustement de la forme des données pour correspondre à l'entrée du modèle
image = np.expand_dims(image, axis=0)  # Ajout d'une dimension pour le batch
text = np.expand_dims(text, axis=0)    # Ajout d'une dimension pour le batch

In [22]:
prediction = model.predict([image, text])

1/1 [==============================] - 4s 4s/step


In [23]:
predicted_class = np.argmax(prediction, axis=1)
predicted_label = le.inverse_transform(predicted_class)


In [24]:
def retourner_decription_categorie(categorie):
        categories = {
            2583: "Accessoires de piscine",
            1560: "Mobilier et décoration pour maison",
            1300: "Modélisme",
            2060: "Accessoires et décorations pour maison",
            2522: "Papeterie",
            1280: "Jouets et peluches enfant",
            2403: "Livres / Magazines en lots",
            2280: "Livres documentaires",
            1920: "Linge de maison",
            1160: "Jeu de cartes à collectionner",
            1320: "Puériculture",
            10: "Littérature étrangère",
            2705: "Littérature historique",
            1140: "Figurines",
            2582: "Accessoires et décorations pour jardin",
            40: "Jeux vidéo neufs",
            2585: "Accessoires et outils de jardinage",
            1302: "Jeux de plein air",
            50: "Accessoires de gaming",
            2462: "Jeux vidéos d’occasion",
            2905: "Jeux vidéo à télécharger",
            60: "Consoles de jeux vidéo",
            2220: "Animalerie",
            1301: "Articles pour nouveaux-nés et bébés",
            1940: "Épicerie",
            1180: "Figurines à peindre et à monter",
            1281: "Jouets et peluches enfant"
        }
        if categorie in categories:
            return categories[categorie]
        else:
            return "Catégorie non trouvée"

In [25]:
retourner_decription_categorie(predicted_label[0])

'Jeux vidéo neufs'